In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from time import sleep
from selenium.webdriver.chrome.options import Options
import pandas as pd
import random

# ヘッドレス、シークレットモード、ユーザーエージェントをランダムで変更
options = Options()
options.add_argument('incognito')
options.add_argument('--headless')
user_agent = [
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.2 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1 Safari/605.1.15',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36']

ua = user_agent[random.randrange(0, len(user_agent), 1)]
options.add_argument('--user-agent=' + ua)

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# 暗黙的待機　要素を取得するまで最大引数秒分待機してね。
# driver.implicitly_wait(10)

url = 'https://code4biz.jp/demo/apparel'
driver.get(url)
sleep(5)

div_elem = driver.find_element(by=By.CSS_SELECTOR, value='#__layout > div > div.StudioCanvas > div.sd > main > section:nth-child(2) > div > div')
items = div_elem.find_elements(by=By.TAG_NAME, value='a')

data = []
item_links = []
# item = items[0]
for item in items:
    item_link = item.get_attribute('href')
    item_links.append(item_link)

for item_link in item_links:
    driver.get(item_link)
    sleep(7)

    elem_detail = driver.find_element(by=By.CSS_SELECTOR, value='#__layout > div > div.StudioCanvas > div.sd > main > div > div:nth-child(2)')
    title = elem_detail.find_element(by=By.TAG_NAME, value='h1').text
    _price = elem_detail.find_elements(by=By.TAG_NAME, value='p')[0].text

    # 数値型にするときはintで括る
    # ValueError: invalid literal for int() with base 10: 'xxx' 10進数の整数に変換できない値がint()関数に渡されたことを伝えている。
    # 待機処理を長くしたら解決した。
    price = int(_price.replace('¥','').replace(',',''))
    elems_detail_li = elem_detail.find_element(by=By.TAG_NAME, value='ul').find_elements(by=By. TAG_NAME, value='li')
    goods_num = int(elems_detail_li[1].find_elements(by=By. TAG_NAME, value='p')[-1].text)
    color = elems_detail_li[2].find_elements(by=By. TAG_NAME, value='p')[-1].text
    category = elems_detail_li[3].find_elements(by=By. TAG_NAME, value='p')[-1].text
    description = elem_detail.find_elements(by=By.TAG_NAME, value='p')[-1].text
    stock_check = elem_detail.find_elements(by=By.TAG_NAME, value='p')[2].text 
    is_stock = True if stock_check == 'カートに入れる' else False

    datum = {
        'title':title,
        'price':price,
        'goods_num':goods_num,
        'color':color,
        'category':category,
        'description':description,
        'stock':is_stock
    }
    data.append(datum)

data_summary = pd.DataFrame(data)
data_summary

In [ ]:
driver.quit()